In [1]:
import librosa
import numpy as np
import os
from keras.models import load_model
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

# Load the saved model
model = load_model('music_genre_classifier_rnn.h5')


In [2]:
# Function to extract features from audio
def extract_features(file_path):
    try:
        audio, _ = librosa.load(file_path, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=22050, n_mfcc=13, duration=30)
        mfccs_processed = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file_path)
        return None 
     
    return mfccs_processed

# Function to load data and extract features
def load_data(data_path):
    labels = []
    features = []
    
    for folder in os.listdir(data_path):
        genre_folder = os.path.join(data_path, folder)
        for file in os.listdir(genre_folder):
            file_path = os.path.join(genre_folder, file)
            feature = extract_features(file_path)
            if feature is not None:
                features.append(feature)
                labels.append(folder)
                
    return np.array(features), np.array(labels)

In [3]:
# Load the test data
data_path = 'Data\genres_original'  # Update with your test dataset path
X_test, y_test = load_data(data_path)

C:\Users\JLCarunungan\AppData\Local\Temp\ipykernel_17084\2025970606.py:4: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, _ = librosa.load(file_path, res_type='kaiser_fast')
c:\Users\JLCarunungan\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\core\audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error encountered while parsing file:  Data\genres_original\jazz\jazz.00054.wav


In [4]:
# Perform predictions
predictions = model.predict(X_test)


# Convert predictions from one-hot encoded to labels
predicted_labels = np.argmax(predictions, axis=1)
true_labels = np.argmax(y_test, axis=0)

32/32 [==============================] - 0s 3ms/step


In [5]:
# Decode labels
predicted_genre = [label_encoder.classes_[label] for label in predicted_labels]
true_genre = [label_encoder.classes_[label] for label in true_labels]

# Print classification report
print(classification_report(true_genre, predicted_genre))

AttributeError: 'LabelEncoder' object has no attribute 'classes_'